# Reto 03-A-Grounding (Fundamentación) 

## 1. Descripción General 

La Fundamentación (Grounding) es una técnica utilizada cuando deseas que el modelo devuelva respuestas fiables a una pregunta dada. A menudo, los modelos GPT necesitarán contexto adicional para proporcionar una respuesta que no alucine, también conocida como dar respuestas falsas. Recuerda que estos modelos GPT solo han sido entrenados con datos hasta septiembre de 2021. Además, los modelos no han sido entrenados con datos específicos de casos de uso.

Existen un par de métodos para llevar a cabo la fundamentación. En este escenario, nos centraremos principalmente en el grounding básico dentro del prompt. En el reto cuatro, verás otras aplicaciones de fundamentación utilizando una base de conocimientos externa e implementando la técnica de Generación Aumentada con Recuperación, o RAG.

Para entender los conceptos básicos de la fundamentación y sus beneficios, este notebook te guiará a través de un ejemplo. A continuación se muestra el escenario que incorporaremos.

## 2. Escenario

Estás escribiendo un informe sobre el torneo de tenis de Wimbledon y necesitas discutir el último partido. Descubre quién fue el ganador de los individuales masculinos y femeninos en 2023.

## 3. Comencemos con la Implementación

Necesitarás importar los módulos necesarios. Las siguientes celdas son pasos clave de configuración que completaste en las tareas anteriores.

In [1]:
import openai
import os
import json

from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

Configura tu entorno para acceder a tus claves de Azure OpenAI. Consulta tu recurso de Azure OpenAI en el Portal de Azure para obtener información sobre tu punto de conexión y tus claves de Azure OpenAI.

Por razones de seguridad, almacena tu información sensible en un archivo .env.

In [2]:
API_KEY = os.getenv("OPENAI_API_KEY")
assert API_KEY, "ERROR: Azure OpenAI Key is missing"
openai.api_key = API_KEY

RESOURCE_ENDPOINT = os.getenv("OPENAI_API_BASE","").strip()
assert RESOURCE_ENDPOINT, "ERROR: Azure OpenAI Endpoint is missing"
assert "openai.azure.com" in RESOURCE_ENDPOINT.lower(), "ERROR: Azure OpenAI Endpoint should be in the form: \n\n\t<your unique endpoint identifier>.openai.azure.com"
openai.api_base = RESOURCE_ENDPOINT

openai.api_type = os.getenv("OPENAI_API_TYPE")
openai.api_version = os.getenv("OPENAI_API_VERSION")

model=os.getenv("CHAT_MODEL_NAME")

### 3.0 Funciones Auxiliares

**get_completion** ayuda a crear una respuesta de OpenAI utilizando el modelo de completado de texto de tu elección.

In [13]:
def get_chat_completion(prompt, model=model):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        engine=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
        max_tokens = 200,
        top_p = 1.0
    )
    return response.choices[0].message["content"]

### 3.1 Antes de la Fundamentación

#### Tarea #1 del Estudiante:

Edita el prompt en la celda siguiente para hacerle una pregunta al modelo sobre el escenario.

Scenario: You are writing a report on the Wimbledon tennis tournament and need to discuss the latest match. Find out who the 2023 winner was for the mens and womens singles.

In [14]:
prompt = f"""
Scenario: You are collecting information about the Wimbledon tennis tournament 2023 for writing a report, and need to find out who was the winner in the latest match for the mens and womens singles. 
So, Who was the winner in the last match of the mens?
"""

model_response = get_chat_completion(prompt, model=model)
print(f"Response: {model_response}\n")


Response: As an AI language model, I do not have access to real-time information. Therefore, I cannot provide you with the latest winner of the Wimbledon tennis tournament 2023. However, you can search for the information on the official website of Wimbledon or other reliable sources.



¿Es esta la respuesta que esperabas?

### 3.2 Después de la Fundamentación

#### Tarea #2 del Estudiante:

Modifica el prompt a continuación para fundamentar el modelo. ¿Cómo puedes obtener una respuesta más precisa que la que recibiste anteriormente?

In [ ]:
context="""
The 2024 Wimbledon Championships will see the best and brightest in the tennis world fight it out for arguably the most prestigious grand slam of them all.
With the tournament being steeped in tradition and history, it is the dream of all tennis players to one day lift the famous trophy.
A list of illustrious champions have graced the courts of SW19 in London, and this year there will be both old challengers and new hopefuls looking to add their names to that roll call.
Twelve months ago, we were blessed with two thrilling finals. The Sporting News looks back at who won the titles last year.
MORE: Full dates and schedule for 2024 Wimbledon Championship
Wimbledon 2023 women's singles champion
Last year's Wimbledon tournament saw history being made, with an unseeded player winning the ladies' singles for the first time ever.
Marketa Vondrousova was ranked at 42 in the world before the tournament began, but pulled off one of the most remarkable runs the championship has ever seen.
After starting her journey with a first-round win over American Peyton Stearns, she followed up with victories against Veronika Kudermetova, Donna Vekic and countrywoman Marie Bouzkova to book a 
spot in the quarterfinal.
A 6-4 2-6 6-4 win over Jessica Pegula was as hard as she was pushed but sent her into the last four, where she dispatched Elina Svitolina in straight sets.
Sixth-seeded Jabeur started the final as the favourite, looking to end her wait for a grand slam after two previous losses on the biggest stage, but she was blown away as Vondrousova won 6-4 6-4 to take 
her place in the record books.
MORE: Who has won the most grand slam titles in history?
Wimbledon 2023 men's singles champion
There was no shortage of drama either in the men's tournament, as Spain's Carlos Alcaraz won the title at the age of just 20, defeating the legendary Novak Djokovic in the final in a thrilling contest.
Spaniard Alcaraz had long threatened to live up to his potential on the biggest stage, and a stunning 1-6 7-6 (8-6) 6-1 3-6 6-4 triumph ended Djokovic's hopes of a record-equalling 24th grand slam.
Alcaraz had looked to be cruising when he recovered from a nervy start to go two sets to one ahead, but Djokovic did what Djokovic does and rallied to win the fourth and then take charge at the start of the fifth.
But the momentum swung again decisively, and Alcaraz finally completed the win after four hours and 43 minutes of a classic.
His triumph marked his first title at the famous All England Club and his ascension as potentially the heir to Djokovic's throne as the best player of his generation. 
Who has won the most Wimbledon tournaments?
In the men's singles tournament, Roger Federer still stands apart as the player with the most Wimbledon titles, having won eight between 2003 and 2017.
Pete Sampras and Novak Djokovic are close behind with seven each.
In the women's tournament, Martina Navratilova has won the most titles in the Open era with nine, while Steffi Graf and Serena Williams both have seven Wimbledon titles to their names.
"""
def get_completion_from_messages(messages, model=model, temperature=0):
    response = openai.ChatCompletion.create(
        engine=model,
        messages=messages,
        temperature=temperature # this is the degree of randomness of the model's output
    )

    return response.choices[0].message["content"]
prompt = f"""
Scenario: You are collecting information about the Wimbledon tennis tournament 2023 for writing a report, and need to find out who was the winner in the latest match for the mens and womens singles. 

STEP 1: Explore for information related to the the Wimbledon tennis tournament 2023 based in this resource : {context}
STEP 2: Analyze how was the last mach of the tournament for the mens and womens singles.
STEP 3: Describe the last match in each case
STEP 4: Determine the answer about the winner of the tournament for the mens and womens singles. 

Provide a answer in the format of a report.

"""
messages =  [      
    {'role':'system', 'content':'You are a copilot who is expert in search information'},
    {'role':'user', 'content':prompt}
]
model_response = get_completion_from_messages(messages, temperature=1)

print(f"Response: {model_response}\n")

Response: Report: Winner of Men's and Women's singles in Wimbledon Tennis Tournament 2023

Introduction:
The Wimbledon Championship is one of the most prestigious Grand Slam tournaments in the tennis world. In 2023, the tournament witnessed some remarkable moments as newcomers took the center stage. In this report, we will look at the winners of the Men's and Women's singles in the tournament.

Women's Singles:
The Women's singles Championship in 2023 saw history being created as an unseeded player, Marketa Vondrousova, won the title for the first time ever. She beat Ons Jabeur, the sixth seed, 6-4, 6-4 in the final, asserting herself as a deserving champion. Vondrousova started as an underdog, ranked 42 in the world before the tournament began, but went on the most significant run in the championship's history to clinch her place in the record books.

Men's Singles:
The men's singles tournament also witnessed an epic showdown between the legendary Novak Djokovic and Carlos Alcaraz, a 

¿Es esta la respuesta que esperabas para ayudarte a escribir tu informe?

## Criterios de Éxito

Para completar este desafío con éxito:

* Demuestra que has comprendido cómo fundamentar un modelo y por qué es importante.
* Asegúrate de obtener una respuesta precisa a tu pregunta que te ayudará a completar el escenario descrito al principio del desafío.
